In [1]:
import numpy as np
import requests
import os
from numpy.linalg import eigh
import matplotlib.pyplot as plt
import mdtraj as md
import nglview as nv
from mpl_toolkits.axes_grid1 import make_axes_locatable


# Working with the Anisotropic Network Model

## Downloading any PDB file

Let's first of all store the PDB id we want to download and the path where the file needs to be located:

In [2]:
pdb_id = '1HIV'
download_path = "./"

And now let's define an auxiliary function to download pdb files to be stored locally in our machine.

In [3]:
def fetch_pdb(pdb_id, download_path="./"):
        url = 'http://files.rcsb.org/download/{}.pdb'.format(pdb_id)
        try:
            res = requests.get(url, allow_redirects=True)
        except:
            print("Could not fetch pdb from {}".format(url))
            return 
        
        file_path = os.path.join(download_path, pdb_id + ".pdb")
        with open(file_path, "wb") as f:
            f.write(res.content)
        print(f"The {pdb_id} file was downloaded succesfully!")

Time to try it!

In [4]:
fetch_pdb(pdb_id, download_path=download_path)

The 1HIV file was downloaded succesfully!


## Parsing the positions of the CA atoms

We are going take an approach not very short and efficient to obtain the coordinates of the CA atoms. But let's do it this way just once:

In [5]:
file_path = os.path.join(download_path, pdb_id + ".pdb")

CA_coordinates = []

with open(file_path) as fff:
    for line in fff.readlines():
        if (line.startswith('ATOM  ') or line.startswith('HETATM')):
            if line.split()[2]=='CA' and line.split()[3]!='1ZK':
          
                x, y, z = line.split()[6:9]
                CA_coordinates.append([float(x), float(y), float(z)])

Ok, `CA_coordinates` is a list of lists... let's convert it to a numpy array:

In [6]:
CA_coordinates = np.array(CA_coordinates)

In [7]:
#Alpha carbon number
len(CA_coordinates)

198

## Defining the equivalent Hessian matrix of the elastic 3D model

In [10]:
def anm_matrix(CA, threshold):
    
    num_CAs = CA.shape[0]
    threshold2 = threshold**2
    
    matrix = np.zeros((3*num_CAs, 3*num_CAs), dtype=float)
    
    ## Sub matrix Hij
    
    for ii in range(num_CAs):
        for jj in range(ii):
                
                rij=CA[ii]-CA[jj]
                distance2=np.dot(rij, rij)
                
                if distance2<threshold2:
                    
                    for kk in range(3):
                        for gg in range(3):
                        
                            val_aux = -(CA[ii][kk]-CA[jj][kk])*(CA[ii][gg]-CA[jj][gg])/distance2
                        
                            matrix[ii+kk,jj+gg]=val_aux
                            matrix[jj+gg,ii+kk]=val_aux
            
            
    ## Sub matrix Hii
    
    for ii in range(num_CAs):
        iii = 3*ii
        for jj in range(num_CAs):
            jjj = 3*jj
            if ii!=jj:
                for kk in range(3):
                    for gg in range(3):
                        matrix[iii+kk, iii+gg] -= matrix[iii+kk, jjj+gg]
                
    return matrix

In [11]:
matrix = anm_matrix(CA_coordinates, 9.0)
matrix

array([[ 0.07350687, -0.81296595, -0.5338413 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.76892089,  0.02597526,  0.76945288, ...,  0.        ,
         0.        ,  0.        ],
       [-0.29590497,  1.58635151,  0.74699934, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

## Diagonalizing from its eigenvalues

In [ ]:
evals, evects = eigh(matrix)

In [ ]:
num_modes = evals.shape[0]

plt.scatter(range(num_modes), evals)
plt.title('Eigenvalues Spectrum')
plt.xlabel('Eigenvalue index')
plt.show()

Ahora los modos normales contienen una componente $x$, $y$ y $z$ para cada CA. ¿Cómo podemos hacer una representación similar a la que hacíamos en el GNM para ver la amplitud participación de cada CA en el modo? ¿Qué tal si representamos algo como $\sqrt{x_{i}^{2} + y_{i}^{2} + z_{i}^{2}}$ para cada CA de índice $i$ en un mismo modo?

In [ ]:
# Esto viene de GNM y debe ser adaptado para ANM
mode_index = 0
plt.plot(range(num_modes), evects[:,mode_index])
plt.title(f'{mode_index}-th normalized mode')
plt.xlabel('CA index')
plt.show()